# Telecom Churn Case Study

## <font color="blue">Problem Statement
> In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecom industries experiences an average of 15% to 25% annual churn rate. Given the fact it costs  5-10 times more to acquire a new customer than to retain a existing customer. So customer retention become more important than customer acquisition.
    
## <font color="blue">Business Objectives
>- For many operators, retaining high profitable customers is the number one business goal.To reduce customer churn, the operators need to predict which customers are high risk of churn.
>- The goal is to build a machine learning model that can able to predict the churning customers using the given dataset.
    

## <font color="blue">Data Understanding and Cleaning:

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Changing settings to view all columns
pd.set_option("display.max_columns",None)

In [3]:
# Reading the data
telecom = pd.read_csv("train.csv", index_col=0)
telecom.head(3)

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,48.58,124.38,1.29,32.24,96.68,2.33,0.0,0.0,0.0,0.0,0.0,0.0,2.23,0.00,0.28,5.29,16.04,2.33,0.0,0.0,0.0,0.00,0.00,0.00,7.53,16.04,2.61,46.34,124.38,1.01,18.75,80.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.09,204.99,1.01,0.0,0.0,0.0,8.20,0.63,0.00,0.38,0.0,0.0,81.21,221.68,3.63,2.43,3.68,7.79,0.83,21.08,16.91,0.00,0.00,0.0,3.26,24.76,24.71,0.00,7.61,0.21,7.46,19.96,14.96,0.0,0.0,0.0,0.0,0.0,0.0,7.46,27.58,15.18,11.84,53.04,40.56,0.0,0.0,0.66,0.0,0.0,0.0,1.11,0.69,0.00,3,2,2,77,65,10,65,65,10,6/22/2014,7/10/2014,8/24/2014,65,65,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.0,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,0.00,0.00,0.00,0.00,25.99,30.89,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,22.01,29.79,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,30.73,31.66,0.00,0.0,0.0,0.00,30.73,31.66,1.68,19.09,10.53,1.41,18.68,11.09,0.35,1.66,3.4,3.44,39.44,25.03,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,3.44,39.44,25.04,0.0,0.0,0.01,0.0,0.

In [4]:
# Checking the dimensions of the dataset
telecom.shape

(69999, 171)

In [5]:
# Checking the informations of the dataset
telecom.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69999 entries, 0 to 69998
Data columns (total 171 columns):
 #    Column                    Dtype  
---   ------                    -----  
 0    circle_id                 int64  
 1    loc_og_t2o_mou            float64
 2    std_og_t2o_mou            float64
 3    loc_ic_t2o_mou            float64
 4    last_date_of_month_6      object 
 5    last_date_of_month_7      object 
 6    last_date_of_month_8      object 
 7    arpu_6                    float64
 8    arpu_7                    float64
 9    arpu_8                    float64
 10   onnet_mou_6               float64
 11   onnet_mou_7               float64
 12   onnet_mou_8               float64
 13   offnet_mou_6              float64
 14   offnet_mou_7              float64
 15   offnet_mou_8              float64
 16   roam_ic_mou_6             float64
 17   roam_ic_mou_7             float64
 18   roam_ic_mou_8             float64
 19   roam_og_mou_6             float64
 20   roam

In [6]:
# Checking the value counts
for index,cols in enumerate(telecom.columns):
    print("\033[1m",str(index+1)+")",cols,"\033[0m")
    print(telecom[cols].value_counts())
    print("\n")

 1) circle_id 
109    69999
Name: circle_id, dtype: int64


 2) loc_og_t2o_mou 
0.0    69297
Name: loc_og_t2o_mou, dtype: int64


 3) std_og_t2o_mou 
0.0    69297
Name: std_og_t2o_mou, dtype: int64


 4) loc_ic_t2o_mou 
0.0    69297
Name: loc_ic_t2o_mou, dtype: int64


 5) last_date_of_month_6 
6/30/2014    69999
Name: last_date_of_month_6, dtype: int64


 6) last_date_of_month_7 
7/31/2014    69600
Name: last_date_of_month_7, dtype: int64


 7) last_date_of_month_8 
8/31/2014    69266
Name: last_date_of_month_8, dtype: int64


 8) arpu_6 
 0.000      2603
 20.000       72
-5.000        72
 1.000        49
 1.500        38
            ... 
 302.334       1
 203.494       1
 45.941        1
 3.420         1
 420.059       1
Name: arpu_6, Length: 61615, dtype: int64


 9) arpu_7 
 0.000      2666
-5.000       107
 20.000       49
 1.000        38
 1.500        37
            ... 
 253.392       1
 266.712       1
 166.851       1
 394.185       1
 139.941       1
Name: arpu_7, Length: 61

0.0    67231
Name: std_og_t2c_mou_6, dtype: int64


 48) std_og_t2c_mou_7 
0.0    67312
Name: std_og_t2c_mou_7, dtype: int64


 49) std_og_t2c_mou_8 
0.0    66296
Name: std_og_t2c_mou_8, dtype: int64


 50) std_og_mou_6 
0.00      20627
0.58         68
0.48         66
0.35         65
0.53         65
          ...  
131.56        1
992.56        1
945.21        1
710.54        1
677.66        1
Name: std_og_mou_6, Length: 21837, dtype: int64


 51) std_og_mou_7 
0.00       21390
0.31          74
0.33          69
0.28          68
0.36          60
           ...  
159.56         1
1035.28        1
749.59         1
87.21          1
62.28          1
Name: std_og_mou_7, Length: 22088, dtype: int64


 52) std_og_mou_8 
0.00       21341
1.01          70
0.66          64
0.50          64
0.58          62
           ...  
223.01         1
1156.14        1
146.74         1
2878.29        1
893.51         1
Name: std_og_mou_8, Length: 21745, dtype: int64


 53) isd_og_mou_6 
0.00      63897
1.01  

0.0    67231
Name: std_ic_t2o_mou_6, dtype: int64


 87) std_ic_t2o_mou_7 
0.0    67312
Name: std_ic_t2o_mou_7, dtype: int64


 88) std_ic_t2o_mou_8 
0.0    66296
Name: std_ic_t2o_mou_8, dtype: int64


 89) std_ic_mou_6 
0.00       16929
0.35         125
0.03         123
0.43         123
0.31         123
           ...  
514.63         1
1207.76        1
155.39         1
872.61         1
228.86         1
Name: std_ic_mou_6, Length: 9876, dtype: int64


 90) std_ic_mou_7 
0.00      17020
0.03        151
0.31        139
0.33        122
0.21        120
          ...  
440.99        1
194.59        1
974.26        1
98.91         1
62.28         1
Name: std_ic_mou_7, Length: 10112, dtype: int64


 91) std_ic_mou_8 
0.00      16478
0.03        127
0.36        121
0.31        118
0.51        117
          ...  
216.68        1
19.15         1
108.13        1
79.94         1
66.46         1
Name: std_ic_mou_8, Length: 9939, dtype: int64


 92) total_ic_mou_6 
0.00       5900
0.18         26
0

0.0     11412
1.0      4394
2.0      1216
3.0       362
4.0       157
5.0       101
6.0        67
7.0        42
8.0        25
9.0        16
10.0       15
11.0       11
12.0       10
14.0        6
15.0        6
13.0        4
18.0        3
20.0        3
17.0        2
16.0        2
24.0        2
22.0        2
21.0        2
19.0        2
23.0        1
34.0        1
31.0        1
Name: count_rech_3g_7, dtype: int64


 133) count_rech_3g_8 
0.0     11624
1.0      4668
2.0      1221
3.0       410
4.0       170
5.0        99
6.0        56
7.0        41
8.0        30
9.0        19
10.0       16
11.0       12
12.0       10
13.0        9
16.0        5
15.0        5
20.0        3
18.0        3
14.0        3
17.0        3
23.0        2
25.0        1
21.0        1
29.0        1
19.0        1
45.0        1
30.0        1
42.0        1
24.0        1
Name: count_rech_3g_8, dtype: int64


 134) av_rech_amt_data_6 
154.00     2586
252.00     1352
198.00      767
25.00       679
98.00       547
           

### Handling Missing values

In [7]:
# Checking for missing values
missing = round((telecom.isnull().sum() / telecom.shape[0])*100,2).sort_values(ascending=False)
print("Total columns that has missing values are",len(missing))
missing[missing !=0]

Total columns that has missing values are 171


count_rech_2g_6             74.90
arpu_2g_6                   74.90
night_pck_user_6            74.90
date_of_last_rech_data_6    74.90
total_rech_data_6           74.90
                            ...  
last_date_of_month_8         1.05
loc_og_t2o_mou               1.00
std_og_t2o_mou               1.00
loc_ic_t2o_mou               1.00
last_date_of_month_7         0.57
Length: 125, dtype: float64

#### <font color="blue">Cleaning date columns

In [8]:
# Extracting the date columns
date_cols = [x for x in telecom.columns if "date" in x]

In [9]:
# Checking the date columns
print(round(telecom[date_cols].isnull().sum() / telecom.shape[0] *100,2))

last_date_of_month_6         0.00
last_date_of_month_7         0.57
last_date_of_month_8         1.05
date_of_last_rech_6          1.57
date_of_last_rech_7          1.76
date_of_last_rech_8          3.52
date_of_last_rech_data_6    74.90
date_of_last_rech_data_7    74.48
date_of_last_rech_data_8    73.69
dtype: float64


In [10]:
# Creating a function to fill mode values
def fill_mode_cols(col_list):
    for cols in col_list:
        mode = telecom[cols].mode()[0]
        telecom[cols].fillna(value=mode, inplace=True)

# Clearing last dates and last rech columns
fill_mode_cols(date_cols[:6])

In [11]:
# Converting date series columns into useful information

# Converting date columns into date time format
telecom[date_cols] = telecom[date_cols].apply(pd.to_datetime)

# Creating recency recharge column
telecom["recent_rech_6"] = telecom["last_date_of_month_6"] - telecom["date_of_last_rech_6"]
telecom["recent_rech_7"] = telecom["last_date_of_month_7"] - telecom["date_of_last_rech_7"]
telecom["recent_rech_8"] = telecom["last_date_of_month_8"] - telecom["date_of_last_rech_8"]

# Creating recency recharge for data column
telecom["recent_rech_data_6"] = telecom["last_date_of_month_6"] - telecom["date_of_last_rech_data_6"]
telecom["recent_rech_data_7"] = telecom["last_date_of_month_7"] - telecom["date_of_last_rech_data_7"]
telecom["recent_rech_data_8"] = telecom["last_date_of_month_8"] - telecom["date_of_last_rech_data_8"]

In [12]:
# Creating a list of recency columns in the dataset
recency_cols = [x for x in telecom.columns if "recent" in x]

# Extracting the dates from the columns
for cols in recency_cols:
    telecom[cols] = telecom[cols].astype(str)
    telecom[cols] = telecom[cols].str.split(" ",expand=True)[0]
    
# Imputing the missing values in the recency cols with zero
for cols in recency_cols:
    telecom.loc[(telecom[cols]=="NaT"), cols] = 0
    
# Dropping the dates columns which is reduntant columns
telecom.drop(date_cols, axis=1, inplace=True)

#### <font color="blue">Cleaning categorical columns

In [13]:
# Extracting the categorical columns
cat_cols = ["churn_probability","fb_user_8","fb_user_7","fb_user_6","night_pck_user_8",
            "night_pck_user_7","night_pck_user_6"]

# Imputing the null values of categorical columns with additional category called (-1)
for cols in cat_cols:
    telecom[cols].fillna(value=-1, inplace=True)

#### <font color="blue">Cleaning numeric columns

In [14]:
# Extracting the numeric columns from the dataset
num_cols = telecom.columns.difference(cat_cols + recency_cols)

# Lets extract the recharge, arpu, other columns from the num_cols
mou_cols = [x for x in num_cols if "mou" in x]
rec_cols = [x for x in num_cols if "rec" in x]
arpu_cols = [x for x in num_cols if "arpu" in x]
other_cols = [x for x in num_cols if "other" in x]

# Imputing ARPU and recharge columns with zero
def zero_imputation(list_col):
    for cols in list_col:
        telecom[cols].fillna(value=0, inplace=True)
        
# Imputing zero for all the remaining columns
zero_imputation(mou_cols)
zero_imputation(rec_cols)
zero_imputation(arpu_cols)
zero_imputation(other_cols)

# Checking the missing values after imputation
print("The total missing values in the dataset after cleaning is",telecom.isnull().sum().sum())

The total missing values in the dataset after cleaning is 0


In [ ]:
# Deriving some useful columns


# Checking for duplicate columns
#def duplicate_cols(df):
    dup_cols = set()
    for x in range(telecom.shape[1]):
        col = df.iloc[:,x]
        for y in range(x+1, telecom.shape[1]):
            other_col = df.iloc[:,y]
            if col.equals(other_col):
                dup_cols.add(telecom.columns[y])
    return len(dup_cols)

print("Number of duplicate columns :",duplicate_cols(telecom))

In [18]:
# Checking for missing values
#round(telecom.isnull().sum() / telecom.shape[0]*100,2)